In [28]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [29]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation,'')
    return text

In [30]:
with open("../static/model/model.pickle","rb") as f:
    model = pickle.load(f)

In [31]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw=file.read().splitlines()

In [32]:
vocab=pd.read_csv("../static/model/vocabulary.txt",header=None)
tokens=vocab[0].tolist()

In [33]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [34]:
def preprocessing(text):
    data=pd.DataFrame([text],columns=['tweet'])
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    data['tweet']=data['tweet'].apply(lambda x:" ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data['tweet']=data['tweet'].apply(remove_punctuations)
    data['tweet']=data['tweet'].str.replace('\\d+', '',regex=True)
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
    data['tweet']=data['tweet'].apply(lambda x:" ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [35]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i]=1

        vectorized_list.append(sentence_list)

    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new

In [48]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction ==1:
        return 'negative'
    else:
        return 'positive'

In [52]:
txt = "awesome work experience"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'